In [7]:
from scrapy import Selector
import requests
import time
import pandas as pd
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
from datetime import datetime as dt

In [8]:
bq_credentials = service_account.Credentials.from_service_account_file(r"D:\work\Big Query Key\brimore-fae8c-b3fdc9b459a2.json")
bq_client = bigquery.Client(credentials=bq_credentials, project=bq_credentials.project_id)


In [9]:
query = '''
    SELECT  distinct concat(device.mobile_brand_name," ",device.mobile_marketing_name) as SearchKey
        FROM `brimore-fae8c.eve_analytics.events_historical` 
        WHERE concat(device.mobile_brand_name," ",device.mobile_marketing_name) IS NOT NULL
'''

In [10]:
query_job = bq_client.query(query)

In [11]:
members_phones = pd.read_gbq(query, project_id=bq_credentials.project_id, credentials=bq_credentials)

ImportError: Missing optional dependency 'pandas-gbq'. pandas-gbq is required to load data from Google BigQuery. See the docs: https://pandas-gbq.readthedocs.io. Use pip or conda to install pandas-gbq.

In [ ]:
members_phones = members_phones.set_index('SearchKey')
key_words = members_phones.index.tolist()

In [ ]:
start_time = time.time()
dict={}
list_urls=[]
dic_list_urls = {}
for key in key_words:
#     print(key)
    url='https://mobizil.com/?s=' + key
    html=requests.get(url).content
    sel=Selector(text=html)
    for i in range(1,11):
        l = sel.xpath('//*[@id="content"]/div/div/div[1]/div/article[%d]/h2/a/text()' % i).extract()
        s = ''.join(n for n in l)
        if s.find('سعر و مواصفات') != -1:
            x = (sel.xpath('//*[@id="content"]/div/div/div[1]/div/article[%d]/h2/a/@href' % i).extract()[0])
            list_urls.append(x)
            dic_list_urls[x] = key
            break
#     dict[url] = key
print(time.time() - start_time)

In [2]:
# dic_list_urls

In [3]:
start_time = time.time()
dictionary2={}
for url, v in dic_list_urls.items():
    html=requests.get(url).content
    sel=Selector(text=html)
    for i in range(1,50):
        l = sel.xpath('//*[@id="aps-overview"]/div[2]/p[%d]/text()' % i).extract()
        s = ''.join(n for n in l)
        if s.find('جنية') !=-1:
            price=(sel.xpath('//*[@id="aps-overview"]/div[2]/p[%d]/text()' % i).extract()[0])
            break
    dictionary2[v] = price
print(time.time() - start_time)

NameError: name 'dic_list_urls' is not defined

In [10]:
phone_price=pd.DataFrame(dictionary2.items(), columns=['SearchKey', 'Price'])

In [11]:
member_phone_price = pd.merge(members_phones,phone_price,how='left',on='SearchKey')

In [12]:
member_phone_price

,SearchKey,Price
0,Alcatel 1C,NaN
1,Alcatel PIXI 4(6),NaN
2,Alcatel OneTouch Pixi 3 (7),NaN
3,TCL Plex,السعر للنسخة 128 جيجا مع 6 جيجا رام حوالي 3400...
4,Huawei Honor Play,NaN
...,...,...
789,LG V30s Thinq,NaN
790,Lava Iris 61,NaN
791,Asus ZenFone 5,NaN
792,Tecno Spark 8T,NaN


In [13]:
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
print('before loading')
    # Load Data
job = bq_client.load_table_from_dataframe(
member_phone_price,'brimore-fae8c.eve_analytics.mobizil_scaping', job_config=job_config)  # Make an API request.
print('after loading')
job.result()  # Wait for the job to complete.

before loading
after loading


LoadJob<project=brimore-fae8c, location=US, id=af065524-68f9-445b-b5ab-f06260e3f313>